<font color = green >

## Home Task 

</font>


The Task is to extract relevant infromation from the messy medical data using regex.

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

1) Extract the date strings
Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

2) Normalize the extracted dates considering the following rules:  
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

3) Sort records in ascending chronological order accoring.




In [107]:
import re
import pandas as pd

In [108]:
data = open("dates.txt", "r").read().splitlines()[:500]

In [109]:
regex = r'\b(?:\d{1,2}/\d{1,2}/(?:\d{2})?\d{2,4}|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[- ,.]? \d{1,2}(?:st|nd|rd|th)?,? \d{2,4}|(?:0[1-9]|1[0-2])/\d{4}|(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?: \d{1,2}[ ,])? (?:\d{2})?\d{2,4})\b'

In [110]:
dates = [re.findall(regex, line) for line in data]

In [111]:
normalized_dates = []
for i, date in enumerate(dates):
    if date:
        date = date[0]
        # Перетворюємо дати формату xx/xx/xx на формат mm/dd/yy
        if re.match(r'\d{1,2}/\d{1,2}/\d{2}$', date):
            date = date[:-2] + '19' + date[-2:]
        # Якщо день відсутній (наприклад, 9/2009), припускаємо, що це перший день місяця (наприклад, 1 вересня 2009 року)
        elif re.match(r'\d{1,2}/\d{4}$', date):
            date = date[:-4] + '01/' + date[-4:]
        # Якщо місяць відсутній (наприклад, 2010), припускаємо, що це перший січня цього року (наприклад, 1 січня 2010 року)
        elif re.match(r'\d{4}$', date):
            date = '01/01/' + date
        normalized_dates.append((pd.to_datetime(date, errors='coerce'), i))
    else:
        normalized_dates.append((None, i))

In [112]:
sorted_dates = sorted((pd.to_datetime(date, errors='coerce'), idx) for date, idx in normalized_dates if date is not None)

In [113]:
df = pd.DataFrame(sorted_dates, columns=['res', 'index'])
df

,res,index
0,1971-04-10,9
1,1971-05-18,84
2,1971-07-08,2
3,1971-07-11,53
4,1971-09-12,28
...,...,...
350,2016-05-01,231
351,2016-05-30,141
352,2016-10-13,186
353,2016-10-19,161
